# Análisis y Reporte sobre el desempeño del modelo

Daniel Salvador Cázares García A01197517

En este breve reporte se hará un análisis acerca del desempeño de un modelo de machine learning implementado con la librería sklearn, que utiliza la técnica de Random Forest Regressor para realizar predicciones de precios de bienes raíces para el dataset "Real estate price prediction". Así mismo, se utilizarán técnicas de regularización y ajuste para intentar mejorar el desempeño.

**Librerías**

In [127]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import GridSearchCV

**Lectura de los datos**

In [128]:
df = pd.read_csv('Real estate.csv', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414 entries, 1 to 414
Data columns (total 7 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   X1 transaction date                     414 non-null    float64
 1   X2 house age                            414 non-null    float64
 2   X3 distance to the nearest MRT station  414 non-null    float64
 3   X4 number of convenience stores         414 non-null    int64  
 4   X5 latitude                             414 non-null    float64
 5   X6 longitude                            414 non-null    float64
 6   Y house price of unit area              414 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 25.9 KB


In [129]:
# Dividir datos en columnas para entrenamiento (X) y para predicción (y)
X = df[['X2 house age', 'X3 distance to the nearest MRT station', 'X4 number of convenience stores', 'X5 latitude', 'X6 longitude']]
y = df['Y house price of unit area']

## Análisis del desempeño

En esta primera parte, se busca analizar el desempeño de la implementación ya realizada. La técnica de Machine Learning seleccionada fue **Random Forest Regressor** con sklearn. El objetivo es predecir el valor de la variable `Y house price of unit area` con base en las variables `X`

### Separación de datos en conjuntos entrenamiento, validación y prueba

El primer paso antes de entrenar el modelo es separar los datos en 3 conjuntos: entrenamiento, validación y prueba. En este caso, los datos se dividieron de la siguiente forma:
* Entrenamiento: 80%
* Validación: 10%
* Prueba: 10%

In [130]:
X_train, X_rest, y_train, y_rest = train_test_split(X,y, train_size=0.8, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state=42)

In [131]:
print("Total de datos: ", len(df))
print("Entrenamiento: ", len(X_train))
print("Validación: ", len(X_val))
print("Prueba: ", len(X_test))

Total de datos:  414
Entrenamiento:  331
Validación:  41
Prueba:  42


### Entrenamiento

Se entreno el modelo con la técnica de Random Forest Regressor

In [132]:
regressor = RandomForestRegressor(n_estimators=100, random_state=42, criterion="squared_error")
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_val)

### Análisis del desempeño

**Score**

In [133]:
regressor.score(X_val, y_val)

0.7515455801398772

Se puede observar que el modelo tiene un score de 0.75 con los datos de validación, lo cual indica que el este tiene un desempeño aceptable. Sin embargo, existe la posibilidad de que este valor sea mejorable al ajustar parametros y transformar los datos.

**Error (MSE y RMSE)**

In [134]:
mse, bias, var = bias_variance_decomp(regressor, X_train.values, y_train.values, X_val.values, y_val.values, loss='mse', num_rounds=200, random_seed=42)

In [135]:
print("MSE: ", mse)
print("RMSE: ", mse**(1/2.0))

MSE:  57.10302988405323
RMSE:  7.556654675453499


**Grado de bias (sesgo) y varianza**

Por lo general, las técnicas de Random Forest deben tener un sesgo bajo y una alta varianza, lo que se traduce en un Overfitting. A continuación se observan las métricas obtenidas para el modelo realizado.

In [136]:
print("Bias: ", bias)
print("Varianza: ", var)

Bias:  41.85468279469873
Varianza:  15.248347089354535


**Nivel de ajuste**

Se puede observar que el bias es considerablemente mayor que la varianza, por lo que puede existir cierto grado de **Underfitting**. Además, ambos valores son elevados, lo que puede provocar que el modelo sea inconsistente y no tenga una buena precisión. Sin embargo, hay que considerar que todavía no se ha hecho escalamiento de los datos, lo cual puede impactar fuertemente a modelos como Random Forest. 

## Mejora del modelo

Como se pudo observar, el primer modelo no obtuvo tan buenos resultados. Por lo tanto, se buscará mejorarlo. Para mejorar los resultados del modelo, existen 2 tareas importantes: preprocesamiento de los datos y ajuste de parámetros.

### Preprocesamiento de datos

**Escalamiento**

Un procedimiento que no se realizo al inicio y que si tiene impacto en algoritmos como regresión lineal o Random Forest es el escalamiento de los datos. A continuación se hará un escalamiento de las variables de entrenamiento con el módulo StandardScaler de sklearn.

In [137]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train) # Entrenamiento
X_val = sc.transform(X_val) # Validación
X_test = sc.transform(X_test) # Prueba

**Reducción de variables**

Antes de pasar al ajuste de parámetros, también se puede observar la importancia que tienen las diferentes columnas en los resultados del modelo. Para de esta forma, remover aquellas que no sean significativas.

In [138]:
feature_imp = pd.Series(regressor.feature_importances_,index=['X2 house age', 'X3 distance to the nearest MRT station', 'X4 number of convenience stores', 'X5 latitude', 'X6 longitude']).sort_values(ascending=False)
feature_imp

X3 distance to the nearest MRT station    0.587487
X2 house age                              0.191782
X5 latitude                               0.129762
X6 longitude                              0.062155
X4 number of convenience stores           0.028814
dtype: float64

Como se puede observar, la variable X4 number of convenience stores tiene poca importancia, por lo que se decidio probar eliminarla para observar cómo se ve afectado el modelo.

In [139]:
# Selección de columnas
X = df[['X2 house age', 'X3 distance to the nearest MRT station', 'X5 latitude', 'X6 longitude']]
y = df['Y house price of unit area']

# División en conjuntos de datos
X_train, X_rest, y_train, y_rest = train_test_split(X,y, train_size=0.8, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state=42)

# Escalamiento
sc = StandardScaler()
X_train = sc.fit_transform(X_train) # Entrenamiento
X_val = sc.transform(X_val) # Validación
X_test = sc.transform(X_test) # Prueba

### Ajuste de hiperpárametros

Ya realizado un preprocesamiento de los datos, ahora se busca encontrar los párametros que permitan mejorar los resultados. Para encontrar los mejores hiperpárametros se uso GridSearchCV, el cual permite entrenar al modelo con distintas combinaciones de párametros para encontrar la combinación más óptima.

Los párametros a probar son los siguientes:

In [140]:
param_grid = {
    'n_estimators': [100, 200, 300, 500, 1000],
    'max_features': [2, 3],
    'max_depth': [10, 50, 100, 200],
}

In [141]:
estimator = RandomForestRegressor()
reg_tuned = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

reg_tuned.fit(X_train, y_train)
reg_tuned.best_estimator_

Fitting 3 folds for each of 40 candidates, totalling 120 fits


RandomForestRegressor(max_depth=200, max_features=3, n_estimators=300)

Se puede observar que los parametros que mejores resultados obtuvieron fueron los anteriores. Ahora ya solo queda volver a entrenar el modelo con los parámetros recomendados y el conjunto de prueba.

In [150]:
newReg = RandomForestRegressor(n_estimators=300, max_features=3, max_depth=200, random_state=42, criterion="squared_error")
newReg.fit(X_train, y_train)
predictions = newReg.predict(X_test)

### Desempeño del nuevo modelo

Ya por último, solo toca comparar las métricas para observar si se logro mejorar el modelo.

**Accuracy**

In [151]:
error = abs(predictions - y_test)
accuracy = 100 - (100 * np.mean(error / y_test))
accuracy

88.79176455301811

El accuracy aumento a 88.79, con lo cual se puede observar que el desempeño aumento de forma considerable.

**MSE y RMSE**

In [162]:
mse, bias, var = bias_variance_decomp(newReg, X_train, y_train.values, X_test, y_test.values, loss='mse', num_rounds=200, random_seed=42)

In [163]:
print("MSE: ", mse)
print("RMSE: ", mse**(1/2.0))

MSE:  28.74700373745699
RMSE:  5.3616232371789225


Se puede ver que el error también disminuyo notoriamente.

**Bias y varianza**

In [164]:
print("Bias: ", bias)
print("Varianza: ", var)

Bias:  20.873299671965142
Varianza:  7.87370406549183


Ya por último, se puede ver que del mismo modo, que el accuracy y el error, el bias y la varianza también mejoraron. Sin embargo, sigue existiendo cierto grado de Underfitting.